This Code will give a basin of attraction for a specified slow-fast coupled oscillator ODE system, it can have N variables. If the Fixed Point attractor is known set FP to the this attractor. If this is unknown, use the eqm solving function to find eqm points.
//
This programme uses Parallel Processing to compute the basin. 

In [1]:
import numpy as np
from scipy.optimize import root
import scipy.integrate as si
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.patches as patches
import matplotlib.colors as colors
from joblib import Parallel, delayed
import itertools as it


In [ ]:
#PARAMETERS OF THE SYSTEM
epsilon = 0.2 #Controls the slow timescale
eta = 10  #fixed parameter of system
alpha = np.pi / 2  #average phase difference of system
kappa = 0 #coupling factor
omega = [-3,-4,-5] #array of natural frequencies
N = len(omega) #number of oscillators
args = (omega, kappa, eta, epsilon, alpha,N) # if any args are added/removed update here

In [ ]:
filename = "BasinMatrix.xlsx"

In [ ]:
#TO SET RANDOM INITIAL CONDITION COMMENT OUT IF NOT NEEDED.
mu0 = np.random.rand()*10 #Change 10 to increase\decrease the range in which the slow variable is in.
phi0 = np.random.rand(N)*np.pi*2
y0 = [mu0,*phi0]

In [ ]:
#PARAMETERS FOR THIS PROGRAMME
threshold = 1e-1 #threshold used to determine attractor type
grid_size = 3 #grid_size X grid_size set of initial conditions for Basin of Attraction (if too large, and not using GPU may take an extremely long time to compute)
y0_range = np.linspace(-5,12,grid_size) #range for time dependent slow timescale parameter
y1_range = np.linspace(0,2*np.pi,grid_size) #range for Osc. 1 (Used in basin computation)

In [ ]:
#TIME PARAMETERS
tf = 20000000 #end of eval
tspan = (0,tf)
te = 19999990 #starting evaluating from te
tlength = 1000  #the number of points being evaluated
t_eval = np.linspace(te,tf,tlength)

In [ ]:
# DEF. FOR SYSTEM OF ODES 
#this includes the time dependency of the system(used in solve(y0) function)
def ode_N(t,y, omega, kappa, N, eta, epsilon, alpha):

    dydt = np.zeros(N + 1)

    for i in range(N):
        dydt[i + 1] = omega[i] + y[0] - np.sin(y[i + 1]) + (kappa / N) * sum(np.sin(y[j] - y[i]) for j in range(N))
    
    dydt[0] = epsilon * (-y[0] + eta - (eta / N) * sum(np.sin(y[j + 1] + alpha) for j in range(N)))
    return dydt

In [ ]:
#DEF. SYSTEM (FOR DETERMINING THE EQUILIBRIUM POINTS)
def equations(y, omega, kappa, N, eta, epsilon, alpha):

    dydt = np.zeros(N + 1)

    for i in range(N):
        dydt[i + 1] = omega[i] + y[0] - np.sin(y[i + 1]) + (kappa / N) * sum(np.sin(y[j] - y[i]) for j in range(N))
    
    dydt[0] = epsilon * (-y[0] + eta - (eta / N) * sum(np.sin(y[j + 1] + alpha) for j in range(N)))
    return dydt

In [ ]:
#Solves system for Eqm points which should correlate to attractors but need to be checked using phase portait. (mu,phi = solve(eqm) and plot_phi_i_v_mu(mu,phi))
#Returns an array of unique equilibrium points rounded to 3 sig. fig.
def equilibriumpoints(equations, N, omega, kappa, eta, epsilon, alpha,size):
    # Generate initial guesses
    mu0 = np.linspace(-30, 10, size)
    phi0 = np.linspace(0, 2 * np.pi, size)
    ICphi = list(it.product(phi0, repeat=N))  # Convert product iterator to list
    
    # Create all combinations of (mu, phi1, phi2, ..., phiN)
    Guesses = np.array([(mu, *phi) for phi in ICphi for mu in mu0])
    
    eqms = []  # Store equilibrium points
    
    # Solve for equilibrium points
    for i in range(Guesses.shape[0]):
        y0 = Guesses[i]
        eqm = root(equations, y0, args=(omega, kappa, N, eta, epsilon, alpha), method='lm')

        if eqm.success:  # Only store successful solutions
            mu = eqm.x[0]
            phi_values = [np.mod(eqm.x[j + 1], 2 * np.pi) for j in range(N)]
            eqms.append((mu, *phi_values))  # Append solution as tuple
    
    # Convert to NumPy array and get unique values
    eqms = np.array(eqms)
    eqms = np.round(eqms, 3)
    unique = np.unique(eqms, axis=0)
    
    return unique


In [ ]:
#Modulates the oscillator trajectories such that the system oscillates between (0,2pi)
def mod2pi(phi):
    phimod = []
    for i in phi:
        i = np.mod(i,2*np.pi)
        phimod = np.append(phimod,i)
    return phimod

In [ ]:
#Plots in a T x R phase space
def plot_phi_v_mu(mu,phi_cleaned):
   
    plt.figure(figsize=(10, 6))
    for i in range(len(phi_cleaned)):
        plt.plot( mu ,phi_cleaned[i], label=f'Phi {i+1}') 
    plt.legend()
    plt.grid()
    plt.xlabel(r'$\mu(t)$')
    plt.ylabel(r'$\varphi_i(t)$')
    plt.title('Dynamics of the System')
    plt.show()

In [ ]:
#Plots the trajectories on a 2-torus projection
def torus_kappa(y0,omega,kappa,N,eta,epsilon,alpha):
    soln = solve_ivp(ode_2, t_span, y0, args=(omega, kappa, N, eta, epsilon, alpha), t_eval=t_eval)
    x = np.mod(soln.y[1],np.pi*2)
    y = np.mod(soln.y[2],np.pi*2)
    u = np.mod(soln.y[3],np.pi*2)
    z = soln.y[0]

    theta1 = x
    theta2 = y
    theta3 = u

    # Torus parameters
    R = 10  # Major radius of the torus
    r = 2  # Minor radius of the torus

    # Map the oscillator phases to torus coordinates
    #Phi1 and Phi2
    z_traj = (R + r * np.cos(theta1)) * np.cos(theta2)
    y_traj = (R + r * np.cos(theta1)) * np.sin(theta2)
    x_traj = r * np.sin(theta1)

    #Phi1 and Phi3
    u_traj = (R + r * np.cos(theta1)) * np.cos(theta3)
    v_traj = (R + r * np.cos(theta1)) * np.sin(theta3)
    w_traj = r * np.sin(theta1)

    #Phi2 and Phi3
    f_traj = (R + r * np.cos(theta2)) * np.cos(theta3)
    g_traj = (R + r * np.cos(theta2)) * np.sin(theta3)
    h_traj = r * np.sin(theta2)

    # Create the torus surface for visualization
    theta = np.linspace(0, 2 * np.pi, 50)
    phi = np.linspace(0, 2 * np.pi, 50)
    theta, phi = np.meshgrid(theta, phi)
    z_torus = (R + r * np.cos(theta)) * np.cos(phi)
    y_torus = (R + r * np.cos(theta)) * np.sin(phi)
    x_torus = r * np.sin(theta)

    # Plot the torus and the trajectory of the oscillators
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(x_torus, y_torus, z_torus, color='grey', alpha=0.05)
    ax.plot(x_traj, y_traj, z_traj, color='k', lw=2, label=r'Oscillator trajectory for $\varphi_1,\varphi_2$,')
    #ax.plot(w_traj, u_traj, v_traj, color='k', lw=2, label=r'Oscillator trajectory for $\varphi_1,\varphi_3$')
    #ax.plot(h_traj, f_traj, g_traj, color='k', lw=2, label=r'Oscillator trajectory for $\varphi_2,\varphi_3$')

    ax.set_title(f'Trajectory of 2 Coupled 2π-Periodic Oscillators on a 2-Torus,k={kappa},N=3,t= {t_span[1]}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()
    plt.savefig(f'PhasePortratitN=3t={t_span[1]}k={kappa}.png',dpi=400)
    plt.show()


In [ ]:
#Plots the trajectories on a T^2 x R phase space 
def plot_T2xR(y0,omega,kappa,N,eta,epsilon,alpha):
    soln = solve_ivp(ode_2, t_span, y0, args=(omega, kappa, N, eta, epsilon, alpha), t_eval=t_eval)
    x = np.mod(soln.y[1],np.pi*2)
    y = np.mod(soln.y[2],np.pi*2)
    u = np.mod(soln.y[3],np.pi*2)
    z = soln.y[0]


    fig = plt.figure(figsize=(15,5))
    
    ax1 = fig.add_subplot(131, projection='3d')
    ax1.scatter(x,y,z,s=0.5,color= 'k')
    ax1.set_xlabel(r'$\varphi_1$')
    ax1.set_ylabel(r'$\varphi_2$')
    ax1.set_zlabel(r'$\mu$')
    ax1.set_title('(a)',fontsize=34)

    ax2 = fig.add_subplot(132, projection='3d')
    ax2.scatter(x,u,z,s=0.5,color= 'k')
    ax2.set_xlabel(r'$\varphi_1$')
    ax2.set_ylabel(r'$\varphi_3$')
    ax2.set_zlabel(r'$\mu$')
    ax2.set_title('(b)',fontsize=34)



    ax3 = fig.add_subplot(133, projection='3d')
    ax3.scatter(y,u,z,s=0.5,color= 'k')
    ax3.set_xlabel(r'$\varphi_2$')
    ax3.set_ylabel(r'$\varphi_3$')
    ax3.set_zlabel(r'$\mu$')
    ax3.set_title('(c)',fontsize=34)

    fig.suptitle(f'k = {kappa}',ha='left')
    plt.tight_layout()
    plt.show()

In [33]:
#Solves the system using solve_ivp function & modulates the trajectory of the oscillators with 2pi
def solve(y0):
    soln2 = si.solve_ivp(ode_N,tspan,y0,args = (omega, kappa, N, eta, epsilon, alpha),method = 'Radau',t_eval=t_eval)
    print(soln2.y)
    phi = []
    for i in range(N):
        p = mod2pi(soln2.y[i,:])
        phi.append(p) 
    mu = soln2.y[0,:]
    return mu,phi

In [ ]:
#classifies the attrator the trajectory is tending to using derivative stability method.
def classify_attractor(solution,threshold):
    state_final = solution.y[:, -1]  # Last state
    derivatives = np.array([ode_N(0, state_final, omega, kappa, N, eta, epsilon, alpha)])  

    # Fixed point check
    if np.linalg.norm(derivatives) < threshold: ##checking how close the system is to having 0 derivatives --> no change in the system => Fixed point
        return 1  # Fixed point
    
    # Periodic behavior check
    else:
        return 2

In [ ]:
#Computes the trajectory of the given initial conditions
def compute_basin_point(y0_0, y1_0,FP,threshold):
    FP =  np.delete(FP,0) 
    initial_conditions = np.insert(FP,0,y0_0,)
    initial_conditions = np.delete(initial_conditions,1) 
    initial_conditions = np.insert(initial_conditions,1,y1_0)
    print(f"y0: {initial_conditions}, shape: {np.shape(initial_conditions)}")
    solution = si.solve_ivp(ode_N,tspan,initial_conditions,args =(omega, kappa, N, eta, epsilon, alpha),method = 'Radau',t_eval=t_eval)
    
    attractor_class = classify_attractor(solution, threshold)
    
    return int(attractor_class)    

In [15]:
#USES PARALLEL PROCESSING TO COMPUTE THE BASIN
def compute_basin(grid_size,threshold,y0_range,y1_range,FP,epsilon):
    num_cores = -1  # Use all available cores
    basin_results = Parallel(n_jobs=num_cores)(delayed(compute_basin_point)(y0, y1,FP,threshold) 
                                               for y1 in y1_range for y0 in y0_range)

    # Convert results into a grid
    basin_map = np.array(basin_results,dtype = int).reshape(grid_size, grid_size)
    return basin_map   

In [ ]:
#Plots the basin of attraction from the matrix computed from compute_basin()
def plot_basin(basin_map,epsilon,y0_range,y1_range,FP,threshold):
    cmap = colors.ListedColormap(["mediumseagreen", "plum"])
    plt.figure(figsize=(16,14))

    plt.imshow(basin_map, origin='lower', extent=[y0_range[0], y0_range[-1], y1_range[0], y1_range[-1]], cmap=cmap) 
    legend_patches = [
        patches.Patch(color="mediumseagreen", label="Fixed Point"),
        patches.Patch(color="plum", label="Limit Cycle")
    ]


    plt.legend(handles=legend_patches, loc="upper right", frameon=True)
    plt.title(r'Basin of Attraction, $\varphi_2$ = $\varphi_{2,EQM}$ ')
    plt.title("Basins of Attraction for Coupled Oscillator System")

    plt.savefig(f'plot_epsilon={epsilon}.png', dpi=300)
    #plt.close()

    plt.show()

In [ ]:
#Saves the basinmatrix to excel file named with filename in parameters.
#basin_map = compute_basin()
def Save_Basin(basin_map,filename):
    df = pd.DataFrame(basin_map)

    # Save to an Excel file
    df.to_excel(filename, index=False, header=False, engine="openpyxl")